In [2]:
import pandas as pd
import numpy as np

In [17]:
def pass_data(csv):
    df = pd.read_csv(csv)
    #print df.head()
    LonLat = df['coordinates'].str[1:-1]     #remove "[]"
    temp_lonlat = LonLat.str.split(",")
    df['lat'] = temp_lonlat.apply(lambda x: x[0]).astype(float)
    df['lon'] = temp_lonlat.apply(lambda x: x[1]).astype(float)
    temp_time = df['time'].str.split(":")
    df['datetime'] = df['date'] + ' ' + df['time']    
    
    df = df.sort_values(['device_id','date','time'], ascending=[1,1,1])
    #print df.columns
    
    
    
    ##========== new_df
    new_df = df[['device_id','s_d0','lon','lat', 'date', 'time']]
    new_df['datetime'] = pd.to_datetime(df['datetime'], format=('%Y-%m-%d %H:%M:%S'))
    
    return new_df

In [18]:
def get_one_station_ts(new_df, Name):  
    one_df = new_df.loc[new_df['device_id'] == Name]
    one_df = one_df[['device_id','s_d0','lon','lat','datetime']]
    ts_df = one_df.set_index('datetime')
    ts_df_mod = ts_df.resample('20T', label = 'right', closed = 'reight').mean()
    ts_df_trun = ts_df_mod.truncate(before='2017-05-15 00:00:00 ', after='2017-05-22 23:59:59')
    
    return ts_df_trun

In [ ]:
def main():
    # load the data
    new_df = pass_data('C:/Users/adsl/Desktop/pm25_airbox_05.csv')
    
    SiteEngName_list = new_df['device_id'].unique().tolist()
    
    appended_data = []   
    null_count = []
    
    for ID in SiteEngName_list:
        data = get_one_station_ts(new_df, ID)
        null_count.append(data['s_d0'].isnull().sum())
    filter_baseline = np.percentile(null_count, 75)
    #print filter_baseline
    
    
    for ID in SiteEngName_list:
        data2 = get_one_station_ts(new_df, ID)
        if data2['s_d0'].isnull().sum() < filter_baseline:
            #print data2['s_d0'].isnull().sum()
            fill_data2 = data2.fillna(data2.interpolate())
            fill_data2 = fill_data2.fillna(fill_data2.bfill())
            #print fill_data2['s_d0'].isnull().sum()
            fill_data2['device_id'] = ID
            appended_data.append(fill_data2) ## store dataframes in list
            
    appended_data = pd.concat(appended_data, axis=0)
    #print appended_data
    
    
    appended_data.to_csv('passdata_airbox.csv')

    
    
if __name__ == "__main__":
    main()

C:\Users\adsl\Anaconda2\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
